### Twitter Dev
1. Connect to redis data store
2. Build dataframe from reading redids datastore
3. Get ticker list
4. create attribute filter, setup stream and log
5. Setup log schedule
6. Run stream

In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
from datetime import date, datetime
import traceback
import os

#Twitter requirements
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
#Get Environment Variables
ACCESS_TOKEN = os.environ['ACCESS_TOKEN'] 
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']


#### Build Company DataFrame

In [3]:
companies = json.loads(REDIS.get('companies').decode())
company_df = pd.DataFrame.from_dict(companies, orient='index')
company_df.index.name = 'Ticker'
company_df.columns=['Company']
#Add code to add ticker symbol
company_df['tweet_ticker']=company_df.index.map(lambda x: '$'+x)

company_df.head()

,Company,tweet_ticker
Ticker,,
AAPL,Apple,$AAPL
FB,Facebook,$FB
GOOG,Google Alphabet C,$GOOG
GOOGL,Google Alphabet A,$GOOGL
AMZN,Amazon,$AMZN


In [4]:
tickers = company_df['tweet_ticker'].tolist()

In [5]:
########### Set up Feature Flag #####################

   
def get_feature_flag(feature):
    all_flags = pd.read_msgpack(REDIS.get("feature_flags"))
    
    try:
        return all_flags.get_value(feature, 'State')
        
    except:
        return 'Flag Not Found, not a valid feature flag'

In [6]:
#Want to create a simple list, but how do I handle the users sub?
attributes = ['created_at',
             'id_str',
             'text',
              'quote_count',
              'reply_count',
              'retweet_count',
              'favorite_count',
              'retweeted',
              'lang',
              ['user','name'],
              ['user','followers_count'],
              ['user','statuses_count'],
              ['user','screen_name']
               ]
def filter_attr(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        if isinstance(element, str):
            output[element]=data[element]
        #Handle Nested Attributes
        else:
            string = str(element[0])+'_'+str(element[1])
            output[string]=data[element[0]][element[1]]
  
    #Need to also add the company name to output dictionary.
    #Add all companies tweet applies to in list
    attached_company = []
    
    for company in tickers:
        if data['text'].find(company) > -1:
            attached_company.append(company[1:])
            
    
    output['Company']=attached_company             
    
    return output

#This is a basic listener that just prints received tweets to stdout.
class TweetListener(StreamListener):
    
    def on_data(self, data):
        try:
            if int(REDIS.get('Data_On')) == 1:
                datajson = json.loads(data)
                filtered = filter_attr(datajson)
                #Check to see if a valid tweet
                if filtered['Company'] and filtered['lang']=='en':

                    print(filtered)
                    #Add counter to count stocks. 
                    REDIS.incr('Twitter_Stock_Count')
                    #---------- Insert to Kinesis Stream --------------
                    if int(get_feature_flag('kinesis_stream_write'))==1:
                        response = kinesis.put_record(StreamName="Twitter_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
                    return True
            
        except Exception as e:
            print(e)

    def on_error(self, status):
        error_string = 'The error code is: ' + repr(status)
        print(error_string)
        #Continue even if there is an error
        #Need to publish the error to the redies error handler in manager
        #Send Start event
        send_event('Twitter', 'Error', error_string)
        #Need to think about if I want to continue running
        return True


#Setup Log

global past_tweet_count
past_tweet_count = 0



#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))
    


#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.now()
    log_time = json_serial(log_time)
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)


#Send the log data to the Redis channel.
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    #now = datetime.datetime.now()
    current_tweet_count = int(REDIS.get('Twitter_Stock_Count'))    
     
    global past_tweet_count
    tweet_count_diff = current_tweet_count - past_tweet_count
    past_tweet_count = current_tweet_count
    
    #Send the log event
    send_log(source='Twitter',current_count = current_tweet_count, count_diff=tweet_count_diff)    
    print('Logged Data')


In [7]:
REDIS.set('Twitter_Stock_Count', 0)

#This handles Twitter authetication and the connection to Twitter Streaming API
tweetlist = TweetListener(api=API(wait_on_rate_limit=True,wait_on_rate_limit_notify=True))
auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
stream = Stream(auth, tweetlist)

#Filters by the ticker names
print('Filtering: ' + str(tickers))
stream.filter(track=tickers, async=True)

#Send Start event
send_event('Twitter', 'Activity', 'Data Source Started')

#Setup Schedule
schedule.clear()
schedule.every(30).seconds.do(log)

#Execute
while True:
    schedule.run_pending()
    #Cancel Schedule if an error occurs. and stop this loop. 
    time.sleep(1)

Filtering: ['$AAPL', '$FB', '$GOOG', '$GOOGL', '$AMZN', '$MSFT', '$BAC', '$BA', '$NFLX', '$JPM', '$TSLA', '$CSCO', '$XOM', '$WFC', '$V', '$JNJ', '$PFE', '$INTC', '$HD', '$C', '$UTX', '$CMCSA', '$TWTR', '$GE', '$UNH', '$PCLN', '$BABA', '$NVDA']
{'created_at': 'Wed Feb 28 18:23:42 +0000 2018', 'id_str': '968914621640867840', 'text': '$TWTR https://t.co/z5YXnjrqRg', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'JAE HOKES', 'user_followers_count': 4888, 'user_statuses_count': 17795, 'user_screen_name': 'JaeHokes', 'Company': ['TWTR']}
{'created_at': 'Wed Feb 28 18:23:42 +0000 2018', 'id_str': '968914622949535744', 'text': '$GE has been dead money a long time.\nIt’s been mismanaged.\nIt also has accounting issues.\n— Steve Weiss, Halftime Report\n#TradersEdge', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'FlowerPower', 'user_followers

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


{'created_at': 'Wed Feb 28 18:23:48 +0000 2018', 'id_str': '968914647041609728', 'text': 'RT @EddyElfenbein: Apple got to $180.62 this morning, which is a new all-time high. The magic number for $AAPL is $197.08. When they hit th…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'vagrantcow', 'user_followers_count': 118, 'user_statuses_count': 34111, 'user_screen_name': 'vagrantcow', 'Company': ['AAPL']}
{'created_at': 'Wed Feb 28 18:23:54 +0000 2018', 'id_str': '968914675671937024', 'text': 'RT @EddyElfenbein: Apple got to $180.62 this morning, which is a new all-time high. The magic number for $AAPL is $197.08. When they hit th…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'BB', 'user_followers_count': 726, 'user_statuses_count': 11558, 'user_screen_name': 'GigsandSwigs', 'Company': ['AAPL']}
{'created_at': 'Wed Feb 28 18:23:59 

Logged Data
{'created_at': 'Wed Feb 28 18:25:55 +0000 2018', 'id_str': '968915180439547904', 'text': '$AAPL 20% off Feb 9 lows, making ATH. https://t.co/JgiQ4hJeu7', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Jeff Macke', 'user_followers_count': 91526, 'user_statuses_count': 49118, 'user_screen_name': 'JeffMacke', 'Company': ['AAPL']}
{'created_at': 'Wed Feb 28 18:26:02 +0000 2018', 'id_str': '968915209002864642', 'text': "RT @HaydenCapital: Just going thru $BIDU's iQiyi IPO docs. First read-thru, this thing's cheap vs. $NFLX: 3.2x sales (at $10BN IPO) vs. $NF…", 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Ryan Marcum', 'user_followers_count': 116, 'user_statuses_count': 1506, 'user_screen_name': 'ryankmarcum', 'Company': ['NFLX']}
Logged Data
{'created_at': 'Wed Feb 28 18:26:15 +0000 2018', 'id_str': '968915265307140096', 't

KeyboardInterrupt: 

...

{'created_at': 'Wed Feb 28 18:50:10 +0000 2018', 'id_str': '968921285928669186', 'text': '$CSCO already 3 bagger.', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Attack Pilot', 'user_followers_count': 960, 'user_statuses_count': 62630, 'user_screen_name': 'NCBenWilliams', 'Company': ['CSCO', 'C']}
{'created_at': 'Wed Feb 28 18:50:13 +0000 2018', 'id_str': '968921298226343936', 'text': '$AAPL 18-Jan-19 160 Puts Traded 2,650 Times for $2.1 Million in Premium. https://t.co/oVJJI0059q https://t.co/YE743EMNUR', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Marketchameleon.com', 'user_followers_count': 633, 'user_statuses_count': 24018, 'user_screen_name': 'MC_OptionTrades', 'Company': ['AAPL']}
{'created_at': 'Wed Feb 28 18:50:30 +0000 2018', 'id_str': '968921368686419968', 'text': '$SGRP $AMZN home services should buy out these guys. 

{'created_at': 'Wed Feb 28 18:52:34 +0000 2018', 'id_str': '968921887983177729', 'text': '@jakscholes @librab103 @BonaireVolt @business My take: $TSLA is currently doing around 800-850 per week - way, way… https://t.co/NPvlYcZRP6', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Wolfgang Pipperger', 'user_followers_count': 167, 'user_statuses_count': 7659, 'user_screen_name': 'WPipperger', 'Company': ['TSLA']}
{'created_at': 'Wed Feb 28 18:53:09 +0000 2018', 'id_str': '968922035945656321', 'text': '@timseymour $ERJ -4.22% on a possibility of new Brazil’ s defense secretary being against $BA US joint venture, I s… https://t.co/cEEe7yQ6RI', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Bernardo Araujo, CFA', 'user_followers_count': 16, 'user_statuses_count': 56, 'user_screen_name': 'Bernard60339426', 'Company': ['BA']}
{'created_at': 

{'created_at': 'Wed Feb 28 18:57:14 +0000 2018', 'id_str': '968923060622442496', 'text': 'RT @jjjinvesting: Bank of America goes ex-dividend tomorrow https://t.co/0dwTOHSUpy $BAC', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'NavigateMarketStorm', 'user_followers_count': 2652, 'user_statuses_count': 106923, 'user_screen_name': 'TheMarketStorm', 'Company': ['BAC', 'BA']}
{'created_at': 'Wed Feb 28 18:57:34 +0000 2018', 'id_str': '968923148224794629', 'text': "RT @TedatACA: Key Republican (@repgregwalden)\n'Regulation coming if Big Tech isn't responsible ...' @axios\n\nhttps://t.co/1iIyjAxyWD $GOOGL…", 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'James Jimmy Cieloha', 'user_followers_count': 917, 'user_statuses_count': 130166, 'user_screen_name': 'jamesjimcie', 'Company': ['GOOG', 'GOOGL']}
{'created_at': 'Wed Feb 28 18:57:34 +000

{'created_at': 'Wed Feb 28 19:00:20 +0000 2018', 'id_str': '968923844651245571', 'text': 'That was my idea! Get Hyundai or Kia to make the cars. Or $TSLA buys them with stock. Its insane $TSLA is doing it… https://t.co/YHgiYWJnLJ', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'BossHogg', 'user_followers_count': 954, 'user_statuses_count': 31770, 'user_screen_name': 'BossHoggHazzard', 'Company': ['TSLA']}
{'created_at': 'Wed Feb 28 19:00:22 +0000 2018', 'id_str': '968923851441635328', 'text': 'https://t.co/ffqrCe0nGN Computer + Internet Connection + Tim Sykes Trading Strategy = Lifestyle Upgrade! $FB $TWTR $STUDY', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Trading123', 'user_followers_count': 61, 'user_statuses_count': 32922, 'user_screen_name': 'PennySilver1983', 'Company': ['FB', 'TWTR']}
{'created_at': 'Wed Feb 28 19:00:24 +

{'created_at': 'Wed Feb 28 19:03:02 +0000 2018', 'id_str': '968924521062379527', 'text': 'We still believe in more upside within the cycle from 2.2016 and $AAPL is calling the move #elliottwave https://t.co/5uFg8Rgnwx', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'Sanjay', 'user_followers_count': 92, 'user_statuses_count': 1783, 'user_screen_name': 'EWFSanjay', 'Company': ['AAPL']}
{'created_at': 'Wed Feb 28 19:03:05 +0000 2018', 'id_str': '968924535683780608', 'text': 'TrueNorth Inc. Lowers Holdings in Citigroup Inc $C https://t.co/bFyN7tYo0B', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'lang': 'en', 'user_name': 'The Ledger Gazette', 'user_followers_count': 162, 'user_statuses_count': 119071, 'user_screen_name': 'ledgerzette', 'Company': ['C']}
{'created_at': 'Wed Feb 28 19:03:06 +0000 2018', 'id_str': '968924540553318402', 'text': 'TrueNorth Inc. Lowe